### Extract Data From Calls & Video Transcripts/Interviews

There is a lot of spoke word out there. Podcasts, interviews, sales calls, phone calls.

It is extremely useful to be able to extract this information and create value with it. We're going to run through an example focusing on the B2B sales use case.

**Small plug:** This is a mini-preview on how I built [Thimble](https://thimbleai.com/) which helps Account Executives pull information from their sales calls. Sign up at https://thimbleai.com/ if you want to check it out or here's the [demo video](https://youtu.be/DIw4rbpI9ic) or see more information on [Twitter](https://twitter.com/GregKamradt)

Plug over, Let's learn! 😈

Through building Thimble I've learned a few tricks to make working with transcripts easier:
1. **Name/Role -** Put the name of each person before their sentence. Bonus points if you have their role/company included too. Example: Greg (Marin Transitions): Hey! How's it going?
2. **System instructions -** Be specific with your system prompt about the role you need you bot to play
3. **Only pull from the call -** Emphasize not to make anything up
4. **Don't make the user prompt -** Abstract away any user prompting necessary with key:value pairs

First let's import our packages

In [4]:
# To get environment variables
import os

# Make the display a bit wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# To split our transcript into pieces
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Our chat model. We'll use the default which is gpt-3.5-turbo
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain

# Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate, # I included this one so you know you'll have it but we won't be using it
    HumanMessagePromptTemplate
)

# To create our chat messages
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [5]:
os.environ['GROQ_API_KEY'] = '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'

Then let's load up our data. This is already a formatted transcript of a mock sales call. 70% of the hard part is getting clean data. Don't under estimate the reward you get for cleaning up your data first!

In [6]:
with open('transcript.txt', 'r') as file:
    content = file.read()

In [7]:
print ("Transcript:\n")
print(content[:215]) # Why 215? Because it cut off at a clean line

Transcript:

**Client:** Hi there, thanks for meeting with me today. I'm looking to get a new website developed for my business. 

**Web Developer:** Hi! It's my pleasure. I'm excited to hear about your project. Could you tell m


Split our documents so we don't run into token issues. Experiment with what chunk size words best for your use case

In [8]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=2000, chunk_overlap=250)
texts = text_splitter.create_documents([content])

In [9]:
print (f"You have {len(texts)} texts")
texts[0]

You have 3 texts


Document(page_content="**Client:** Hi there, thanks for meeting with me today. I'm looking to get a new website developed for my business. \n\n**Web Developer:** Hi! It's my pleasure. I'm excited to hear about your project. Could you tell me a bit about your business and what you're looking for in your new website?\n\n**Client:** Sure. I run a boutique travel agency that specializes in custom, luxury travel packages. We cater to high-end clients who are looking for personalized experiences. The current website we have is quite outdated and doesn't really convey the level of luxury and personalization we offer. I want something that feels more modern, elegant, and user-friendly.\n\n**Web Developer:** That sounds like an exciting project. To get a better understanding, can you describe some specific features or elements you want on the new site?\n\n**Client:** Absolutely. First off, I want a visually stunning homepage with high-quality images that immediately showcase the type of experie

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [12]:
# Your api key should be an environment variable, or else put it here
# We are using a chat model in case you wanted to use gpt4
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [29]:
template="""

You are a helpful assistant that helps to extract the requirements that are mentioned in the input.
Your goal is to extract all the key requirements for the website development project. Categorize the requirements into sections such as Design, Features. If you don't know, say, "I don't know"
Directly give the requirements in clear points and don't write any other thing.
list the requirements without including the introductory line like "Here are the requirements categorized into sections:"
dont use any special character or symbol in the output


"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template="{text}" # Simply just pass the text as a human message
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [30]:
chat_prompt = ChatPromptTemplate.from_messages(messages=[system_message_prompt, human_message_prompt])

In [31]:
chain = chat_prompt | llm

# Because we aren't specifying a combine prompt the default one will be used

In [32]:
output = chain.invoke({
                    "text": texts,
                   })

In [35]:
print (output.content)

Design:
modern and elegant design
clean lines
elegant typography
minimalist design
brand guidelines to be followed
color palette and fonts as per brand guide
high-quality images on homepage
subtle animations on homepage
visually stunning homepage

Features:
directory for destinations with detailed pages
each destination page to have detailed information, images, and client testimonials
dedicated section for client testimonials
comprehensive contact form with fields for travel dates, destinations of interest, budget, and special requests
integration with CRM system
integration with email marketing tool
links to social media profiles
content management system for easy updates
user-friendly CMS for updating content

Functionality:
booking and inquiries through contact form
personalized consultations based on client preferences

Technical:
integration with CRM system
integration with email marketing tool
integration with social media platforms
content management system for easy updates
use

In [10]:
!pip install langchain-groq

                                              0.0/84.9 kB ? eta -:--:--
     ---------------------------------------- 84.9/84.9 kB 5.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Awesome! Now we have a bullet point format without needing to have the user specify any additional information.

If you wanted to productionize this you would need to add additional prompts to extract other information from the calls that may be helpful to a sales person. Example: Key Points + Next Steps from the call. You should also parallelize the map calls if you do the map reduce method.

Have other ideas about how something like this could be used? Send me a tweet or DM on [Twitter](https://twitter.com/GregKamradt) or contact@dataindependent.com